In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import networkx as nx
import osmnx as ox
from shapely.geometry import Point, LineString, Polygon
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore")

In [2]:
def network_setting(network):
    _nodes_removed = len([n for (n, deg) in network.out_degree() if deg ==0])
    network.remove_nodes_from([n for (n, deg) in network.out_degree() if deg ==0])
    for component in list(nx.strongly_connected_components(network)):
        if len(component)<10:
            for node in component:
                _nodes_removed+=1
                network.remove_node(node)
    for u, v, k, data in tqdm(G.edges(data=True, keys=True),position=0):
        if 'maxspeed' in data.keys():
            speed_type = type(data['maxspeed'])
            if (speed_type==str):
                if len(data['maxspeed'].split(','))==2:
                    data['maxspeed']=float(data['maxspeed'].split(',')[0])                  
                elif data['maxspeed']=='signals':
                    data['maxspeed']=35.0 # drive speed setting as 35 miles
                else:
                    data['maxspeed']=float(data['maxspeed'].split()[0])
            else:
                data['maxspeed']=float(data['maxspeed'][0].split()[0])
        else:
            data['maxspeed']= 35.0 #miles
        data['maxspeed_meters'] = data['maxspeed']*26.8223 # convert mile to meter
        data['time'] = float(data['length'])/ data['maxspeed_meters']
    print("Removed {} nodes ({:2.4f}%) from the OSMNX network".format(_nodes_removed, _nodes_removed/float(network.number_of_nodes())))
    print("Number of nodes: {}".format(network.number_of_nodes()))
    print("Number of edges: {}".format(network.number_of_edges()))
    return(network)

def hospital_setting(hospitals, G):
    hospitals['nearest_osm']=None
    for i in tqdm(hospitals.index, desc="Find the nearest osm from hospitals", position=0):
        hospitals['nearest_osm'][i] = ox.get_nearest_node(G, [hospitals['Y'][i], hospitals['X'][i]], method='euclidean') # find the nearest node from hospital location
    print ('hospital setting is done')
    return(hospitals)

In [19]:
%%time
#G = ox.load_graphml('Illinois_Network.graphml', folder="Data", node_type=str)
#G = network_setting (G)
G = ox.load_graphml('Data/Illinois_Prepared.graphml', edge_dtypes={"osmid":str,"time":float})
hospitals = gpd.read_file('./Data/HospitalData/{}_Hospital_Info.shp'.format("Illinois"))
hospitals = hospital_setting(hospitals, G)
distances=[10,20,30]

Find the nearest osm from hospitals: 100%|███████████████████████████████████████████| 183/183 [01:25<00:00,  2.15it/s]

hospital setting is done
Wall time: 4min 2s


In [5]:
def dijkstra_cca(G, nearest_osm, distance, distance_unit = "time"):
    road_network = G.subgraph(nx.single_source_dijkstra_path_length(G, nearest_osm, distance, distance_unit))  
    nodes = [Point((data['x'], data['y'])) for node, data in road_network.nodes(data=True)]
    polygon = gpd.GeoSeries(nodes).unary_union.convex_hull ## to create convex hull
    polygon = gpd.GeoDataFrame(gpd.GeoSeries(polygon)) ## change polygon to geopandas
    polygon = polygon.rename(columns={0:'geometry'}).set_geometry('geometry')
    return polygon.copy(deep=True)

In [6]:
def ego_cca(G, nearest_osm, distance, distance_unit = "time"):
    road_network = nx.ego_graph(G, nearest_osm, distance, distance=distance_unit) 
    nodes = [Point((data['x'], data['y'])) for node, data in road_network.nodes(data=True)]
    polygon = gpd.GeoSeries(nodes).unary_union.convex_hull ## to create convex hull
    polygon = gpd.GeoDataFrame(gpd.GeoSeries(polygon)) ## change polygon to geopandas
    polygon = polygon.rename(columns={0:'geometry'}).set_geometry('geometry')
    return polygon.copy(deep=True)

In [20]:
%%time
ego_cca(G, hospitals['nearest_osm'][0], distances[2])

Wall time: 802 ms


,geometry
0,"POLYGON ((-90.08384 38.60189, -90.08789 38.601..."


In [21]:
%%time 
dijkstra_cca(G, hospitals['nearest_osm'][0], distances[2])

Wall time: 232 ms


,geometry
0,"POLYGON ((-90.08384 38.60189, -90.08789 38.601..."


In [18]:
G.adj[236487434]

AdjacencyView({236378002: {0: {'osmid': '21954891', 'name': 'Rock Springs Drive', 'highway': 'residential', 'oneway': False, 'length': 27.953, 'maxspeed': '35.0', 'maxspeed_meters': '938.7805', 'time': '0.029775863473943058'}}, 236487495: {0: {'osmid': '21945565', 'name': 'Memorial Drive', 'highway': 'residential', 'oneway': False, 'length': 380.01199999999994, 'geometry': <shapely.geometry.linestring.LineString object at 0x00000232720C1A00>, 'maxspeed': '35.0', 'maxspeed_meters': '938.7805', 'time': '0.40479323974028003'}}, 236548860: {0: {'osmid': '21954891', 'name': 'Rock Springs Drive', 'highway': 'residential', 'oneway': False, 'length': 363.786, 'geometry': <shapely.geometry.linestring.LineString object at 0x00000232720C17C0>, 'maxspeed': '35.0', 'maxspeed_meters': '938.7805', 'time': '0.38750911421786033'}}})

In [29]:
%%time
ego = []
for i in range(0,100):
    ego.append(ego_cca(G, hospitals['nearest_osm'][i], distances[2]))

Wall time: 3min 5s


In [30]:
%%time
dij = []
for i in range(0,100):
    dij.append( dijkstra_cca(G, hospitals['nearest_osm'][i], distances[2]))

Wall time: 50.2 s


In [10]:
for i in range(10):
    print(ego[i].geometry == dij[i].geometry)

0    True
Name: geometry, dtype: bool
0    True
Name: geometry, dtype: bool
0    True
Name: geometry, dtype: bool
0    True
Name: geometry, dtype: bool
0    True
Name: geometry, dtype: bool
0    True
Name: geometry, dtype: bool
0    True
Name: geometry, dtype: bool
0    True
Name: geometry, dtype: bool
0    True
Name: geometry, dtype: bool
0    True
Name: geometry, dtype: bool
